# Statistical Analysis
## 1. statistical analysis of the factors
description of each factors, make sure all the data are usable and the outliers can be explained 
## 2. correlation anaylsis of factors
there is no strong correlation between any pair of factors
## 3. univariate sorting base on downside risk factor and volatility
both downside risk and vol factors perform pretty well
## 4. bivariate sorting (control for bond types and other factors)
downside risk factor's performance is more robust

In [12]:
#导入数据
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
from scipy.stats import f
import scipy.stats as stats
# additional packages
from statsmodels.stats.diagnostic import lillifors
from pylab import mpl
from scipy import stats
from scipy.stats import norm

data_address = '~/Downloads/'
monthly_cb_value = pd.read_csv(data_address + 'monthly_cb_value_fin_adjReturnStata190901.csv')

## Table 1-A   Statistical Anlysis of Factors
Give a simple description of the major factors in data

In [13]:
def description(a):
    des = pd.DataFrame(np.zeros((1, 14)))
    des.columns = ['mean', 'median', 'var', 'std', 'skew', 'kurt', 'min', 'max', '1st', '5th', '25th', '75th', '95th', '99th']
    des['mean'][0] = a.mean()
    des['median'][0] = a.median()
    des['var'][0] = a.var()
    des['std'][0] = a.std()
    des['skew'][0] = a.skew()
    des['kurt'][0] = a.kurt()
    des['min'][0] = a.min()
    des['max'][0] = a.max()
    des['1st'][0] = a.quantile(0.01)
    des['5th'][0] = a.quantile(0.05)
    des['25th'][0] = a.quantile(0.25)
    des['75th'][0] = a.quantile(0.75)
    des['95th'][0] = a.quantile(0.95)
    des['99th'][0] = a.quantile(0.99)
    
    return des

descrip = pd.DataFrame(np.zeros((1, 14)))

descrip.columns = ['mean', 'median', 'var', 'std', 'skew', 'kurt', 'min', 'max', '1st', '5th', '25th', '75th', '95th', '99th']
descrip_index = ['returns', 'downside_risk_3', 'status_dsr', 'vol', 'skew','illiq_2_2', 'default_spread', 'creditrate', 'res_day', 'coupon', 'total_size']

for n in descrip_index:
    descrip = descrip.append(description(monthly_cb_value[n]))

descrip = descrip[1:]

descrip.index = descrip_index


In [14]:
descrip
#目前return还没有化成百分比

,mean,median,var,std,skew,kurt,min,max,1st,5th,25th,75th,95th,99th
returns,0.002192,0.004757,0.000571,0.023886,-6.108449,43.054608,-0.179702,0.039195,-0.179702,-0.014646,0.002825,0.006723,0.019217,0.039195
downside_risk_3,0.005677,0.000000,0.000879,0.029647,9.376352,112.894635,0.000000,0.873396,0.000000,0.000000,0.000000,0.000000,0.022697,0.190841
status_dsr,-0.004486,-0.004508,0.000005,0.002204,-0.463931,-0.711295,-0.009887,-0.001112,-0.009887,-0.008269,-0.006075,-0.002710,-0.001463,-0.001112
vol,0.001605,0.000000,0.000027,0.005177,5.579493,35.105956,0.000000,0.039332,0.000000,0.000000,0.000000,0.000645,0.007947,0.039332
skew,0.033211,0.000000,4.710706,2.170416,-0.015242,0.852875,-4.619601,4.690416,-4.619601,-4.472136,0.000000,0.000000,4.469533,4.690416
illiq_2_2,0.069389,0.055556,0.013547,0.116393,1.221392,18.985435,-0.999998,1.000000,-0.254995,-0.015029,0.050000,0.071429,0.259467,0.500000
default_spread,1.229138,-1.270753,99.207179,9.960280,5.146521,32.591518,-6.990800,94.651642,-5.287200,-3.967564,-2.481489,0.110372,21.070892,59.087924
creditrate,4.642002,5.000000,2.068598,1.438262,-1.723986,3.628973,0.000000,6.000000,0.000000,0.000000,4.000000,6.000000,6.000000,6.000000
res_day,4.128499,4.136986,5.548658,2.355559,0.030897,0.217219,-2.336717,9.852055,-2.336717,0.394506,2.561009,5.747945,8.534247,9.852055
coupon,5.534797,5.700000,3.319477,1.821943,0.164317,-0.869864,2.540000,10.000000,2.610000,2.930000,3.780000,6.900000,8.500000,10.000000


In [ ]:
#观察return出现极值的情况
ex = pd.DataFrame(monthly_cb_value[monthly_cb_value['returns'] > 1].cb_id.value_counts().index)
ex.columns = ['id']

bondExample = monthly_cb_value[monthly_cb_value['cb_id'] == ex.iloc[0].id] #随机选取这些特殊债券列表中的一只债券
bondExample = bondExample.set_index('datetime').sort_index()
plt.plot(bondExample.close)
plt.title('Time Series Price of a Returns Outlier')
plt.show()
#我们看到这个极值债券的价格走势在18年出现了瞬间的下跌（该极值应该被剔除？）

In [ ]:
#从default spread可以看到其对应的最大值过高， 因此看一下default spread较高的债券的价格走势情况是否正常
ex = pd.DataFrame(monthly_cb_value[monthly_cb_value['default_spread'] > 90].cb_id.value_counts().index)
ex.columns = ['id']

bondExample = monthly_cb_value[monthly_cb_value['cb_id'] == ex.iloc[1].id] #随机选取这些特殊债券列表中的一只债券
bondExample = bondExample.set_index('datetime').sort_index()
plt.plot(bondExample.close)
plt.title('Time Series Price of a Default Spread Outlier')
plt.show()
#下图可以看到该债券的价格出现断崖式下跌，这在我们的预期中是相对正常的（数据本身没有出现问题）

In [ ]:
#为了进一步看Default Spread(同时也是反映yield)是否可以使用，我们统计一下default spread的分布
plt.hist(monthly_cb_value['default_spread'], bins='auto')
plt.title('Bonds Distribution in Default Spread')
plt.show()
#分布中有一部分极值的情况（大于60），从上述分析中我们可以看到该情况起因是债券价格突然降低（可以考虑标准化）

## Table 1-B The Correlation Analysis of Major Factors

In [ ]:
corr_index = ['returns', 'downside_risk_3', 'status_dsr', 'vol', 'skew','illiq_2_2', 'default_spread', 'creditRate', 'res_day']
monthly_cb_value[corr_index].corr()
#相关性基本符合预期 其中vol和downside risk之间的相关性较高，在Univariate Sort和Bivariate Sort过程中需要注意

## Table 2 Univariate Portfolios  Sorted by Downside Risk and Vol


选取债券本身的特征：
1.债券发行量
2.债券种类
3.债券评级
4.债券上市日期
下面是对于上述债券特征的统计描述

In [ ]:
from sort_functions import return_decile, return_decile_2
#these are univariate sorting functions, see sort_functions.py 

In [ ]:
univariate_index = ['returns_n', 'downside_risk_3', 'status_dsr', 'vol', 'skew','illiq_2_2', 'default_spread', 'creditRate', 'res_day']
univSort_downside = return_decile_2('downside_risk_3', monthly_cb_value[monthly_cb_value['downside_risk_3'] != 0], univariate_index)
univSort_downside
#重点看returns_n(下一期收益的变化)可以看到上升趋势非常明显

In [ ]:
univariate_index = ['returns_n', 'downside_risk_3', 'status_dsr', 'vol', 'skew','illiq_2_2', 'default_spread', 'creditRate', 'res_day']
univSort_vol = return_decile_2('vol', monthly_cb_value[monthly_cb_value['downside_risk_3'] != 0], univariate_index)
univSort_vol
#returns_n上升趋势相对明显

## Table 3 Bivariate Sorted Portfolio( Different Bond Types) 

Bivariate Sort 分为两个部分：   
**1. control variable 取值不连续**  
选取债券本身的特征：    
1.债券发行量  
2.债券种类  
3.债券评级  
4.债券上市日期  
先分别统计这些指标的特征，再对债券进行分类，观察这些指标下债券的对于Downside Risk等的portfolio构建结果是否存在一定趋势  
  
**2. control variable 取值连续**  
以Downside_risk, vol为自变量，分析控制债券回报率（returns）, default_spread, illiquidity, total_size, maturity下这两个变量的表现

### 1. control variable 取值不连续

In [ ]:
#分析债券本身的特征（上述四个特征）的统计数据, 然后分析对应的结果
#特征1：bond size
plt.hist(monthly_cb_value['total_size'], bins = 'auto')
plt.title('Bonds Distribution in Total Size')
plt.show()

In [ ]:
monthly_cb_value.loc[monthly_cb_value.total_size.isin(range(0, 15)), 'size_type']='low'
monthly_cb_value.loc[monthly_cb_value.total_size.isin(range(15, 50)), 'size_type']='med'
monthly_cb_value.loc[monthly_cb_value.total_size>50, 'size_type']='high'
monthly_cb_value['size_type'].value_counts()

In [ ]:
from bivariate_sort import bivariate_sort_decile, bivariate_sort_types

In [ ]:
bivariate_sort_types('downside_risk_3', 'size_type', monthly_cb_value[monthly_cb_value['downside_risk_3']!=0], 5)
#从结果来看三种类型中downside_risk和预期收益率都有明显的相关趋势

In [ ]:
bivariate_sort_types('vol', 'size_type', monthly_cb_value[monthly_cb_value['vol']!=0], 5)
#从结果来看三种类型中vol和预期收益率都有一定相关趋势，但是和downside risk相比该趋势较弱（尤其在’high‘类别上）

In [ ]:
#特征2：bond type
monthly_cb_value['bond_type'].value_counts().sort_index()

In [ ]:
bivariate_sort_types('downside_risk_3', 'bond_type', monthly_cb_value[monthly_cb_value['downside_risk_3']!=0], 5)
#后几个类别数据量过少，因此结果为零（正常），几个主要类别的downside risk均有较强趋势，尤其是类别1.0（公司债）

In [ ]:
bivariate_sort_types('vol', 'bond_type', monthly_cb_value[monthly_cb_value['vol']!=0], 5)
#和downside risk相比，vol只有在公司在上有比较明显的相关趋势

In [ ]:
#特征3：credit rate
monthly_cb_value['creditRate'].value_counts().sort_index()

In [ ]:
bivariate_sort_types('downside_risk_3', 'creditRate', monthly_cb_value[monthly_cb_value['downside_risk_3']!=0], 5)

In [ ]:
bivariate_sort_types('vol', 'creditRate', monthly_cb_value[monthly_cb_value['vol']!=0], 5)

In [ ]:
#特征4： 上市时间（on_market_date）
monthly_cb_value['on_market_date'] = monthly_cb_value['on_market_date'].apply(lambda x: pd.to_datetime(x))
monthly_cb_value['on_market_year'] = monthly_cb_value['on_market_date'].apply(lambda x: x.year)
monthly_cb_value['on_market_month'] = monthly_cb_value['on_market_date'].apply(lambda x: x.month)
monthly_cb_value['on_market_year'].value_counts().sort_index()                                                                            

In [ ]:
bivariate_sort_types('downside_risk_3', 'on_market_year', monthly_cb_value[monthly_cb_value['downside_risk_3']!=0], 5)

In [ ]:
bivariate_sort_types('vol', 'on_market_year', monthly_cb_value[monthly_cb_value['vol']!=0], 5)

### 2. control variable 取值连续

In [ ]:
#returns
bivariate_sort_decile('downside_risk_3', 'returns', monthly_cb_value[monthly_cb_value['downside_risk_3']!=0], 5, 5)
#still promising

In [ ]:
bivariate_sort_decile('vol', 'returns', monthly_cb_value[monthly_cb_value['vol']!=0], 5, 5)